## Pandas (Handling Missing Data)

Normalmente en el mundo real, los datos raramente están limpios y son homogéneos. En general, los datasets tendrán datos perdidos.

Para hacerlo más complicado, diferentes fuentes de datos tendrán diferentes tipos de datos perdidos

Nos referiremos a los datos perdidos como NaN, NA o Null.

#### Trade-Offs in Missind Data Conventions

Se han desarrollado varios esquemas para representar datos perdidos en una tabla o DataFrame. Generalmente, giran en torno a dos estrategias:

* Usar una máscara (mask)
* Usar un centinela (sentinel value)


Máscara: es un array separado de los valores, de tipo booleano, que indica el null status de un valor.  Esto supone un sobrecoste en almacenaje y computación

Sentinel Value: es un valor que tiene un patrón raro, del tipo -9999 o similar, que indica que este valor está perdido. Por convención se suele usar un missing floating point value, que forma parte de la especificación IEEE y que se nombra como NaN (not a number)

Dependiendo del lenguaje de programación, la estrategia a seguir difiere en uno u otro caso.

In [5]:
import numpy as np
import pandas as pd # librería de pandas

In [6]:
pd.__version__ #versión de pandas

'0.23.0'

### Missing Data in Pandas

Pandas usa para missing data la estrategia del sentinel, y en particular, dos tipos de valores:

* NaN value
* None Object

#### None Object

Es un objeto de Python que se usa frecuentemente para missing data en el código. Como es un tipo de dato Objeto sólo puede ser usado en Numpy arrays cuando se define un objeto de este tipo.

In [25]:
vals1 = np.array([1, None, 3, 4])

In [26]:
vals1 # dtype de tipo objeto

array([1, None, 3, 4], dtype=object)

In [27]:
type(vals1)

numpy.ndarray

In [28]:
vals1.dtype #tipo objeto

dtype('O')

In [29]:
vals2 = np.array([1, 2, 3, 4])

In [30]:
vals2

array([1, 2, 3, 4])

In [31]:
vals2.dtype #tipo entero

dtype('int64')

El coste de definir un array de tipo objeto puede ser grande, comparado con los tipos de datos nativos, como refleja el ejemplo siguiente

In [32]:
for dtype in ['object', 'int']:
    print("dtype=", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object
38.5 ms ± 719 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype= int
1.17 ms ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



También, en el caso de realizar agregaciones sobre el array, cuando este es de tipo Object, dará un error

In [33]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

#### NaN (Not a Number) Missing Numerical Data

Es una representación de missing data, a través de un valor centinela, que es reconocido por todos los sistemas que usan el IEEE floating point representation standard.

In [34]:
vals3 = np.array([1, np.nan,3, 4])


In [35]:
vals3

array([ 1., nan,  3.,  4.])

In [36]:
vals3.dtype

dtype('float64')

In [37]:
1 + vals3

array([ 2., nan,  4.,  5.])

Hay que tener cuidado con los NaN porque son como una especie de virus, ya que infecta cualquier objeto que toca. Todas las operaciones aritméticas donde participe NaN, dará lugar a NaN

In [39]:
1 + np.nan

nan

In [40]:
0 * np.nan

nan

In [38]:
vals3.sum()

nan

Numpy proporciona algunas agregaciones especiales que ignora estos
valores perdidos

In [42]:
np.nansum(vals3)

8.0

#### NaN and None in Pandas

Ambos tienen su lugar en Pandas, ya que este está diseñado para manejar ambos tipos, intercambiándolos y convirtiéndolos cuando sea conveniente 

In [45]:
pd.Series([1, np.nan, 2, None]) # convierte None a NaN

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Pandas directamente hace casting automáticamente cuando lo ve 
necesario. En este caso un array de tipo entero, al cambiar uno
de los valores a None o NaN automáticamente lo cambia a typo float64

In [46]:
x = pd.Series(range(2), dtype =int)

In [47]:
x

0    0
1    1
dtype: int64

In [49]:
x[0] = None

In [51]:
x #ahora el array es de tipo float64

0    NaN
1    1.0
dtype: float64

#### Upcasting conventions

    TypeClass|Conversion when Storing NAs|NA Sentinel Value
    floating|No change|np.nan
    object|No change|np.nan or None
    integer|Cast to float64|np.nan
    boolean|Cast to object|np.nan or None 

Recuerda que los datos de tipo string se almacenan como tipo objeto

### Operating on Null Values

Pandas usa una serie de métodos para detectar, eliminar y reemplazar null values en los objetos Series y DataFrame:

* isnull() indica si existen valores nulos
* notnull() lo contrario a isnull()
* dropna() devuelve una versión filtrada de los datos eliminando NaN
* fillna() devuelve una copia de los datos reemplazando los valores NaN 

#### Detectando valores nulos

Para ello usamos isnull() y notnull(). Aplica tanto a Series como a DataFrames

In [60]:
data = pd.Series([1, np.nan, 'hello', None])

In [61]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [62]:
data.isnull() # devuelve un array booleano por cada uno de los valores

0    False
1     True
2    False
3     True
dtype: bool

In [63]:
data.notnull() #devuelve un array booleano 

0     True
1    False
2     True
3    False
dtype: bool

In [64]:
data[data.isnull()] #filtra aquellos que son nulos

1     NaN
3    None
dtype: object

In [65]:
data[data.notnull()] #filtra aquellos que no son nulos

0        1
2    hello
dtype: object

### Eliminando valores nulos

Para ello usamos dropna()

In [67]:
data.dropna() # para una Serie, elimina los valores nulos

0        1
2    hello
dtype: object

En un DataFrame no se pueden eliminar valores concretos, por lo que
tendremos que eliminar o bien toda la fila o toda la columna

In [114]:
df = pd.DataFrame([[1, np.nan ,2],
                  [2, 3 ,5],
                  [np.nan, 4,6]])

In [115]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [116]:
df.dropna() # por defecto, elimina por filas

,0,1,2
1,2.0,3.0,5


In [117]:
df.dropna(axis='columns') # elimina a través de las columnas

,2
0,2
1,5
2,6


In [118]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [120]:
df.dropna(axis=1) # elimina a través de las columnas

,2
0,2
1,5
2,6


Estas formas de eliminar información vistas son demasiado salvajes, 
ya que elimina bastante información que es buena y valiosa. Dropna permite ir un poco más allá con dos argumentos adicionales:

* how
* thresh

How:
Con how, podemos decirle cuando eliminar la fila o columna
how='any'  cualquier fila o columna conteniendo un NaN será eliminada
how='all'  todos los valores de la fila o columna tienen que ser NaN

Thresh:
Podemos indicar cual el máximo de NaN que podemos tener en una
fila o columna


In [80]:
df[3] = np.nan

In [81]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [84]:
df.dropna(axis='columns', how='any') 
#elimina aquellas columnas que algún valores es NaN

,2
0,2
1,5
2,6


In [85]:
df.dropna(axis='columns', how='all') 
#elimina aquellas columnas que todos sus valores son NaN

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [87]:
df.dropna(axis='columns', thresh=3) 
# elimina aquellas columnas con 3 o más NaN

,2
0,2
1,5
2,6


### Reemplazando valores nulos

Para ello usamos fillna().
En ocasiones, preferimos reemplazar los valores nulos con otros valores
que o bien puede ser 0, una media o una interpolación de valores correctos

* Valor entero
* Forward-fill (valor previo)
* Back-fill (valor anterior)


In [88]:
data = pd.Series([1, np.nan, 2, 3, None])

In [89]:
data

0    1.0
1    NaN
2    2.0
3    3.0
4    NaN
dtype: float64

In [90]:
# reemplaza con zeros
data.fillna(0)

0    1.0
1    0.0
2    2.0
3    3.0
4    0.0
dtype: float64

In [93]:
# forward-fill: reemplaza con el valor previo
data.fillna(method='ffill')

0    1.0
1    1.0
2    2.0
3    3.0
4    3.0
dtype: float64

In [94]:
# back-fill: reemplaza con el valor anterior
data.fillna(method='bfill')

0    1.0
1    2.0
2    2.0
3    3.0
4    NaN
dtype: float64

Para DataFrames las opciones son similares, pudiendo jugar también 
con los ejes

In [95]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [96]:
df.fillna(0)

,0,1,2,3
0,1.0,0.0,2,0.0
1,2.0,3.0,5,0.0
2,0.0,4.0,6,0.0


In [97]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [107]:
df.fillna(method='ffill', axis=0)

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [108]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
